# PageRank on Flights Dataset

In this project the network (graph) anlysis is used to determine the importance of US airports during August 2017. In this model the graph vertices are airports and the graph edges are the direct flight segments that conect them. 

Given the airport network, we want to answer the quation of which airports are most critical to disruption of the overall network?

In this project Markov chain analysis is being used to answer this quation.

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.sparse
import warnings
from IPython.display import display, Markdown

print(f"- Numpy version:  {np.__version__}")
print(f"- scipy version:  {sp.__version__}")
print(f"- pandas version: {pd.__version__}")

- Numpy version:  1.19.2
- scipy version:  1.4.1
- pandas version: 1.1.3


# Part 0: Reading and exploring data

In [2]:
airport_codes=pd.read_csv('./Data/L_AIRPORT_ID.csv', sep='#')
display(airport_codes.head())
print(airport_codes.shape)

,Code,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"
3,10005,"Little Squaw, AK: Little Squaw Airport"
4,10006,"Kizhuyak, AK: Kizhuyak Bay"


(6350, 2)


In [3]:
flights=pd.read_csv('./Data/US-flights-2017-08.csv')
flights=flights.iloc[:,:-1]
display(flights.head())
print(flights.shape)

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,DEST_AIRPORT_ID,DEST_CITY_MARKET_ID
0,2017-08-01,AA,559,11057,31057,14100,34100
1,2017-08-01,AA,559,15304,33195,11057,31057
2,2017-08-01,AA,560,11298,30194,11278,30852
3,2017-08-01,AA,560,14107,30466,11298,30194
4,2017-08-01,AA,561,14107,30466,13204,31454


(510451, 7)


Each row of the above dataframe is a direct flight that left an origin (ORIGIN_AIRPORT_ID) and arrived at a destination (DEST_AIRPORT_ID) during August 2017. The airport IDs can be found in airport_codes dataframe read earlier.

In [4]:
# for example the airport code for Los Angeles International airport is:
airport_codes[airport_codes['Description'].str.contains('Los Angeles.*International')]

,Code,Description
2765,12892,"Los Angeles, CA: Los Angeles International"


In [5]:
LAX_ID=airport_codes[airport_codes['Description'].str.contains('Los Angeles.*International')]['Code'].iloc[0]
ATL_ID=airport_codes[airport_codes['Description'].str.contains('Hartsfield-Jackson')]['Code'].iloc[0]
print(f"{ATL_ID} : ATL -- {airport_codes[airport_codes['Code']==ATL_ID]['Description'].iloc[0]}")
print(f"{LAX_ID} : LAX -- {airport_codes[airport_codes['Code']==LAX_ID]['Description'].iloc[0]}")

10397 : ATL -- Atlanta, GA: Hartsfield-Jackson Atlanta International
12892 : LAX -- Los Angeles, CA: Los Angeles International


In [6]:
# construct 'flights_ATL_to_LAX':
flights_ATL_to_LAX=flights[(flights["ORIGIN_AIRPORT_ID"]==ATL_ID) & (flights["DEST_AIRPORT_ID"]==LAX_ID)]
display(flights_ATL_to_LAX.head(10))
print("The number of direct flights from ATL to LAX:", len(flights_ATL_to_LAX))

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,DEST_AIRPORT_ID,DEST_CITY_MARKET_ID
21,2017-08-01,AA,1249,10397,30397,12892,32575
668,2017-08-01,AA,2437,10397,30397,12892,32575
1940,2017-08-01,DL,110,10397,30397,12892,32575
2041,2017-08-01,DL,370,10397,30397,12892,32575
2673,2017-08-01,DL,1125,10397,30397,12892,32575
2682,2017-08-01,DL,1133,10397,30397,12892,32575
2734,2017-08-01,DL,1172,10397,30397,12892,32575
2794,2017-08-01,DL,1218,10397,30397,12892,32575
2840,2017-08-01,DL,1255,10397,30397,12892,32575
3052,2017-08-01,DL,1427,10397,30397,12892,32575


The number of direct flights from ATL to LAX: 586


In [7]:
# construct 'flights_LAX_TO_ATL':
flights_LAX_to_ATL=flights[(flights['ORIGIN_AIRPORT_ID']==LAX_ID) & (flights['DEST_AIRPORT_ID']==ATL_ID)]
display(flights_LAX_to_ATL)
print("The number of direct flights from LAX to ATL:", len(flights_LAX_to_ATL))

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,DEST_AIRPORT_ID,DEST_CITY_MARKET_ID
22,2017-08-01,AA,1249,12892,32575,10397,30397
340,2017-08-01,AA,1116,12892,32575,10397,30397
1934,2017-08-01,DL,101,12892,32575,10397,30397
1967,2017-08-01,DL,174,12892,32575,10397,30397
2692,2017-08-01,DL,1140,12892,32575,10397,30397
...,...,...,...,...,...,...,...
500180,2017-08-31,AA,1116,12892,32575,10397,30397
502723,2017-08-31,F9,1480,12892,32575,10397,30397
503545,2017-08-31,WN,193,12892,32575,10397,30397
503546,2017-08-31,WN,1114,12892,32575,10397,30397


The number of direct flights from LAX to ATL: 585


In [8]:
# number of (origin, destination) pairs:
segments=flights.groupby(['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID'], as_index=False).count()
segments=segments.iloc[:,:3]
segments.rename(columns={'FL_DATE':'FL_COUNT'},inplace=True)
segments.head()

,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,FL_COUNT
0,10135,10397,77
1,10135,11433,85
2,10135,13930,18
3,10140,10397,93
4,10140,10423,4


In [9]:
# find the origins and number of outgoing flights
origins=segments[['ORIGIN_AIRPORT_ID', 'FL_COUNT']].groupby('ORIGIN_AIRPORT_ID', as_index=False).sum()
origins.rename(columns={'FL_COUNT':'ORIGIN_COUNT'}, inplace=True)
origins.head()

,ORIGIN_AIRPORT_ID,ORIGIN_COUNT
0,10135,180
1,10140,1761
2,10141,62
3,10146,41
4,10154,176


To get an idea of what airport are likely to be more important using Markov chain analysis, let's rank airports by the total number of outgoing segments (flights originate at the airport).

In [10]:
# The top 10 airports with the most outgoing flights are:
origins_top10=origins.sort_values(by='ORIGIN_COUNT', ascending=False).reset_index(drop=True)
origins_top10=origins_top10.merge(airport_codes,left_on='ORIGIN_AIRPORT_ID',right_on='Code').drop('Code', axis=1)[:10]
origins_top10.rename(columns={'ORIGIN_AIRPORT_ID':'ID', 'ORIGIN_COUNT':'Count'},inplace=True)
display(origins_top10)

,ID,Count,Description
0,10397,31899,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern..."
1,13930,25757,"Chicago, IL: Chicago O'Hare International"
2,11292,20891,"Denver, CO: Denver International"
3,12892,19399,"Los Angeles, CA: Los Angeles International"
4,14771,16641,"San Francisco, CA: San Francisco International"
5,11298,15977,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter..."
6,14747,13578,"Seattle, WA: Seattle/Tacoma International"
7,12889,13367,"Las Vegas, NV: McCarran International"
8,14107,13040,"Phoenix, AZ: Phoenix Sky Harbor International"
9,13487,12808,"Minneapolis, MN: Minneapolis-St Paul Internati..."


In [11]:
dests=segments[['DEST_AIRPORT_ID','FL_COUNT']].groupby('DEST_AIRPORT_ID', as_index=False).sum()
dests.rename(columns={'FL_COUNT':'DEST_COUNT'},inplace=True)
display(dests.head())

# The top 10 destination airports are:
dests_top10=dests.sort_values(by='DEST_COUNT',ascending=False).reset_index(drop=True)[:10]
dests_top10=dests_top10.merge(airport_codes,left_on='DEST_AIRPORT_ID',right_on='Code').drop('Code',axis=1)
dests_top10.rename(columns={'DEST_AIRPORT_ID':'ID','DEST_COUNT':'Count'}, inplace=True)
dests_top10

,DEST_AIRPORT_ID,DEST_COUNT
0,10135,179
1,10140,1763
2,10141,62
3,10146,40
4,10154,176


,ID,Count,Description
0,10397,31901,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern..."
1,13930,25778,"Chicago, IL: Chicago O'Hare International"
2,11292,20897,"Denver, CO: Denver International"
3,12892,19387,"Los Angeles, CA: Los Angeles International"
4,14771,16651,"San Francisco, CA: San Francisco International"
5,11298,15978,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter..."
6,14747,13582,"Seattle, WA: Seattle/Tacoma International"
7,12889,13374,"Las Vegas, NV: McCarran International"
8,14107,13039,"Phoenix, AZ: Phoenix Sky Harbor International"
9,13487,12800,"Minneapolis, MN: Minneapolis-St Paul Internati..."


# Part 1: Constructing the state-transition matrix

In [12]:
n_airports=len(airport_codes)
print(f"There are {n_airports} airports")

There are 6350 airports


In [13]:
origin_indecies=airport_codes[['Code']].rename(columns={'Code':'ORIGIN_AIRPORT_ID'})
origin_indecies['ORIGIN_INDEX']=airport_codes.index
origin_indecies.head(10)

,ORIGIN_AIRPORT_ID,ORIGIN_INDEX
0,10001,0
1,10003,1
2,10004,2
3,10005,3
4,10006,4
5,10007,5
6,10008,6
7,10009,7
8,10010,8
9,10011,9


In [14]:
dest_indecies=airport_codes[['Code']].rename(columns={'Code':'DEST_AIRPORT_ID'})
dest_indecies['DEST_INDEX']=airport_codes.index
dest_indecies.head(10)

,DEST_AIRPORT_ID,DEST_INDEX
0,10001,0
1,10003,1
2,10004,2
3,10005,3
4,10006,4
5,10007,5
6,10008,6
7,10009,7
8,10010,8
9,10011,9


In [15]:
segments=segments.merge(origin_indecies, on='ORIGIN_AIRPORT_ID')
segments=segments.merge(dest_indecies, on='DEST_AIRPORT_ID')
segments.head()

,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,FL_COUNT,ORIGIN_INDEX,DEST_INDEX
0,10135,10397,77,119,373
1,10140,10397,93,124,373
2,10146,10397,41,130,373
3,10158,10397,31,142,373
4,10185,10397,12,168,373


In [16]:
total_flights=segments[['ORIGIN_INDEX', 'FL_COUNT']].groupby('ORIGIN_INDEX',as_index=False).count()
total_flights.rename(columns={'FL_COUNT':'OUTDEGREE'}, inplace=True)
total_flights.head()

,ORIGIN_INDEX,OUTDEGREE
0,119,3
1,124,23
2,125,1
3,130,1
4,138,3


In [17]:
segments=segments.merge(total_flights, on='ORIGIN_INDEX')
segments.head()

,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,FL_COUNT,ORIGIN_INDEX,DEST_INDEX,OUTDEGREE
0,10135,10397,77,119,373,3
1,10135,11433,85,119,1375,3
2,10135,13930,18,119,3770,3
3,10140,10397,93,124,373,23
4,10140,13930,41,124,3770,23


In [18]:
segments["WEIGHTS"]=1.0/segments["OUTDEGREE"]
segments.head()

,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,FL_COUNT,ORIGIN_INDEX,DEST_INDEX,OUTDEGREE,WEIGHTS
0,10135,10397,77,119,373,3,0.333333
1,10135,11433,85,119,1375,3,0.333333
2,10135,13930,18,119,3770,3,0.333333
3,10140,10397,93,124,373,23,0.043478
4,10140,13930,41,124,3770,23,0.043478


Now we have the number of outgoing flights from each point and the probability of going from one airport to the other one. We can construct the state-transition matrix P. The matrix would be n_airports X n_airports matrix with most of elements equal to zero. Thus, it is more efficient to use sparse matrix from scipy.spars

The rows in Matrix P are origin airports and the columns are destination airports and the values are the probability for an agent to go from origin to desitination at that point Pij.

In [19]:
P=scipy.sparse.coo_matrix((segments["WEIGHTS"], (segments["ORIGIN_INDEX"],segments["DEST_INDEX"])),
                          shape=(n_airports,n_airports))

# Part 2: Computing distribution and Ranking airports

suppose the agent is equally likely at any airport with outgoing flight. So the initial state of the agent is a vector of equal possibility to be in a airport with outgoing flight

In [20]:
x0=np.zeros(n_airports)
x0[segments['ORIGIN_INDEX']]=1
x0/=x0.sum()

Given the state_transition matrix P, an initial vector x0, and the number of time steps t_max, we can compute markov_chain 

In [21]:
def eval_morkov_chain(P, x0, t_max):
    for _ in range(t_max):
        x0=P.T.dot(x0)
    return x0

In [22]:
# let's find the steady-state after 50 time steps:

T_MAX=100
x=eval_morkov_chain(P, x0, T_MAX)

In [23]:
r=np.argsort(-x)[:10]
airport_top10=airport_codes.iloc[r].copy()

airport_top10['Rank']=np.arange(1,11)
airport_top10['x(t)']=x[r]

airport_top10

,Code,Description,Rank,x(t)
373,10397,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",1,0.037384
3770,13930,"Chicago, IL: Chicago O'Hare International",2,0.036043
1245,11292,"Denver, CO: Denver International",3,0.031214
3347,13487,"Minneapolis, MN: Minneapolis-St Paul Internati...",4,0.026761
2177,12266,"Houston, TX: George Bush Intercontinental/Houston",5,0.024809
1250,11298,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",6,0.024587
1375,11433,"Detroit, MI: Detroit Metro Wayne County",7,0.024483
3941,14107,"Phoenix, AZ: Phoenix Sky Harbor International",8,0.021018
4646,14869,"Salt Lake City, UT: Salt Lake City International",9,0.020037
1552,11618,"Newark, NJ: Newark Liberty International",10,0.019544


We can compare the two rankings side_by_side where the first ranking is based on Markov chain and the other one is solely based on number of flights

In [24]:
origin_top10_ranking=origins_top10[["ID", "Description"]].copy()
origin_top10_ranking["Rank"]=np.arange(1,11)
origin_top10_ranking.rename(columns={"ID":"Code", "Description":"Description"}, inplace=True)

airport_top10_compare=pd.merge(airport_top10.drop('x(t)',axis=1), origin_top10_ranking, how='outer', 
                               on='Code', suffixes=['_MC','_SG'])
airport_top10_compare

,Code,Description_MC,Rank_MC,Description_SG,Rank_SG
0,10397,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",1.0,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",1.0
1,13930,"Chicago, IL: Chicago O'Hare International",2.0,"Chicago, IL: Chicago O'Hare International",2.0
2,11292,"Denver, CO: Denver International",3.0,"Denver, CO: Denver International",3.0
3,13487,"Minneapolis, MN: Minneapolis-St Paul Internati...",4.0,"Minneapolis, MN: Minneapolis-St Paul Internati...",10.0
4,12266,"Houston, TX: George Bush Intercontinental/Houston",5.0,NaN,NaN
5,11298,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",6.0,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",6.0
6,11433,"Detroit, MI: Detroit Metro Wayne County",7.0,NaN,NaN
7,14107,"Phoenix, AZ: Phoenix Sky Harbor International",8.0,"Phoenix, AZ: Phoenix Sky Harbor International",9.0
8,14869,"Salt Lake City, UT: Salt Lake City International",9.0,NaN,NaN
9,11618,"Newark, NJ: Newark Liberty International",10.0,NaN,NaN


# Conclusion

The Morkov chain analysis determined the most critical airports to disruption of the airline network. 

At the begining it might be expected that the importance of airports is related to whether an airport has most inbound and outgoing flights. However, if there are multiple routes between two airports, then even if one of the routes includes a high-degree airport in its path, an agent can go on an alternate route and reach the destination. So having a high-degree is not the only contributor to how important an airport is.